In [1]:
import pandas as pd
import json
import os
import numpy as np
import re #regex 

In [2]:
dfs = []
for r, d, f in os.walk('input'):
    for file in f:
        if 'withheldtweets.json' in file:  # alt: if ‘control' in file:
            dfs.append(pd.read_json('%s/%s' % (r, file), lines=True))

df_cen = pd.concat(dfs)
#df_cen = df_cen.dropna(subset=['withheld_in_countries'])

In [3]:
worthKeeping = ["text", "truncated", "user",
                "withheld_in_countries", "entities", "lang",
                "possibly_sensitive", "extended_tweet"]
df_cen = df_cen[worthKeeping]

In [4]:
df_cen

,text,truncated,user,withheld_in_countries,entities,lang,possibly_sensitive,extended_tweet
0,RT @ZaidZamanHamid: کرونا ایسے تباہی پھیلا رہا...,False,"{'id': 1124646527895576579, 'id_str': '1124646...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",ur,NaN,NaN
1,RT @ZaidZamanHamid: کرونا ایسے تباہی پھیلا رہا...,False,"{'id': 1124646527895576579, 'id_str': '1124646...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",ur,NaN,NaN
2,RT @sunochanda12: جسکو ہمارے ملک کے لبدلز معصو...,False,"{'id': 421049995, 'id_str': '421049995', 'name...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",ur,NaN,NaN
3,"RT @ZaidZamanHamid: If it is blood clotting, w...",False,"{'id': 300795161, 'id_str': '300795161', 'name...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,NaN,NaN
4,"RT @ZaidZamanHamid: If it is blood clotting, w...",False,"{'id': 300795161, 'id_str': '300795161', 'name...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,NaN,NaN
...,...,...,...,...,...,...,...,...
300,RT @bidemitweets: Dem Ina Ina Ina Ina Ina I kn...,False,"{'id': 1339162661813149697, 'id_str': '1339162...",[XX],"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,NaN,NaN
301,RT @HizbAmerica: Help the Muslims of Sri Lanka...,False,"{'id': 1086171998802362370, 'id_str': '1086171...",[RU],"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,NaN,NaN
302,RT @HizbAmerica: Help the Muslims of Sri Lanka...,False,"{'id': 1086171998802362370, 'id_str': '1086171...",[RU],"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,NaN,NaN
303,RT @emrecelikrumi: .@JoeBiden in case you miss...,False,"{'id': 126758373, 'id_str': '126758373', 'name...",[TR],"{'hashtags': [{'text': 'Erdogan', 'indices': [...",en,NaN,NaN


In [5]:
#replace the NaN with coherent values to make further processing easier
df_cen['possibly_sensitive'] = df_cen['possibly_sensitive'].fillna(0.0)
df_cen

,text,truncated,user,withheld_in_countries,entities,lang,possibly_sensitive,extended_tweet
0,RT @ZaidZamanHamid: کرونا ایسے تباہی پھیلا رہا...,False,"{'id': 1124646527895576579, 'id_str': '1124646...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",ur,0.0,NaN
1,RT @ZaidZamanHamid: کرونا ایسے تباہی پھیلا رہا...,False,"{'id': 1124646527895576579, 'id_str': '1124646...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",ur,0.0,NaN
2,RT @sunochanda12: جسکو ہمارے ملک کے لبدلز معصو...,False,"{'id': 421049995, 'id_str': '421049995', 'name...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",ur,0.0,NaN
3,"RT @ZaidZamanHamid: If it is blood clotting, w...",False,"{'id': 300795161, 'id_str': '300795161', 'name...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,0.0,NaN
4,"RT @ZaidZamanHamid: If it is blood clotting, w...",False,"{'id': 300795161, 'id_str': '300795161', 'name...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,0.0,NaN
...,...,...,...,...,...,...,...,...
300,RT @bidemitweets: Dem Ina Ina Ina Ina Ina I kn...,False,"{'id': 1339162661813149697, 'id_str': '1339162...",[XX],"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,0.0,NaN
301,RT @HizbAmerica: Help the Muslims of Sri Lanka...,False,"{'id': 1086171998802362370, 'id_str': '1086171...",[RU],"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,0.0,NaN
302,RT @HizbAmerica: Help the Muslims of Sri Lanka...,False,"{'id': 1086171998802362370, 'id_str': '1086171...",[RU],"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,0.0,NaN
303,RT @emrecelikrumi: .@JoeBiden in case you miss...,False,"{'id': 126758373, 'id_str': '126758373', 'name...",[TR],"{'hashtags': [{'text': 'Erdogan', 'indices': [...",en,0.0,NaN


In [6]:
#recover the full text for truncated tweets

dfRaw = df_cen.values
for line in dfRaw:
    if not pd.isna(line[-1]):
        line[0] = line[-1]["full_text"]
        
    line[0] = re.sub(r'http\S+', '', line[0])
    #remove urls from tweets
    #they are shortened anyway so we can't make use of them

dfRaw = np.delete(dfRaw, len(worthKeeping)-1, axis=1) #remove "extended_tweet"
worthKeeping.remove("extended_tweet")

dfRaw = np.delete(dfRaw, 1, axis=1) #remove "truncated"
worthKeeping.remove("truncated")

In [7]:
#extract hashtags seperately

for line in dfRaw:
    line[3] = [x["text"] for x in line[3]["hashtags"]]
worthKeeping[3] = "hashtags"

In [8]:
#create a feature for user-verified and user-followers_count

verified = [line[1]["verified"] for line in dfRaw]
followers = [line[1]["followers_count"] for line in dfRaw]
location = [line[1]["location"] for line in dfRaw]

dfRaw = np.c_[dfRaw, verified, followers, location]
worthKeeping += ["verified_account", "followers_count", "location"]

In [9]:
#binary feature for whether the tweet has been withheld anywhere

withheld = []
for line in dfRaw:
    if not isinstance(line[2], list):
        line[2] = []
    withheld.append(len(line[2]) != 0)
        
dfRaw = np.c_[dfRaw, withheld]
worthKeeping += ["withheld_anywhere"]

In [10]:
df_cen = pd.DataFrame(dfRaw, columns = worthKeeping)
df_cen

,text,user,withheld_in_countries,hashtags,lang,possibly_sensitive,verified_account,followers_count,location,withheld_anywhere
0,RT @ZaidZamanHamid: کرونا ایسے تباہی پھیلا رہا...,"{'id': 1124646527895576579, 'id_str': '1124646...",[IN],[],ur,0.0,False,4,None,True
1,RT @ZaidZamanHamid: کرونا ایسے تباہی پھیلا رہا...,"{'id': 1124646527895576579, 'id_str': '1124646...",[IN],[],ur,0.0,False,4,None,True
2,RT @sunochanda12: جسکو ہمارے ملک کے لبدلز معصو...,"{'id': 421049995, 'id_str': '421049995', 'name...",[IN],[],ur,0.0,False,371,"Islamabad, Pakistan",True
3,"RT @ZaidZamanHamid: If it is blood clotting, w...","{'id': 300795161, 'id_str': '300795161', 'name...",[IN],[],en,0.0,False,2471,لاہور، پاکستان - دبئی ، امارت,True
4,"RT @ZaidZamanHamid: If it is blood clotting, w...","{'id': 300795161, 'id_str': '300795161', 'name...",[IN],[],en,0.0,False,2471,لاہور، پاکستان - دبئی ، امارت,True
...,...,...,...,...,...,...,...,...,...,...
42318,RT @bidemitweets: Dem Ina Ina Ina Ina Ina I kn...,"{'id': 1339162661813149697, 'id_str': '1339162...",[XX],[],en,0.0,False,661,Abuja,True
42319,RT @HizbAmerica: Help the Muslims of Sri Lanka...,"{'id': 1086171998802362370, 'id_str': '1086171...",[RU],[],en,0.0,False,23,"Mombasa, Kenya",True
42320,RT @HizbAmerica: Help the Muslims of Sri Lanka...,"{'id': 1086171998802362370, 'id_str': '1086171...",[RU],[],en,0.0,False,23,"Mombasa, Kenya",True
42321,RT @emrecelikrumi: .@JoeBiden in case you miss...,"{'id': 126758373, 'id_str': '126758373', 'name...",[TR],"[Erdogan, Israel]",en,0.0,False,18411,Sydney》Brisbane》Melboune》DC》NY,True


In [11]:
cleanCols = filter(lambda x: x != "user", worthKeeping)
df_clean = df_cen[cleanCols]

In [12]:
df_clean

,text,withheld_in_countries,hashtags,lang,possibly_sensitive,verified_account,followers_count,location,withheld_anywhere
0,RT @ZaidZamanHamid: کرونا ایسے تباہی پھیلا رہا...,[IN],[],ur,0.0,False,4,None,True
1,RT @ZaidZamanHamid: کرونا ایسے تباہی پھیلا رہا...,[IN],[],ur,0.0,False,4,None,True
2,RT @sunochanda12: جسکو ہمارے ملک کے لبدلز معصو...,[IN],[],ur,0.0,False,371,"Islamabad, Pakistan",True
3,"RT @ZaidZamanHamid: If it is blood clotting, w...",[IN],[],en,0.0,False,2471,لاہور، پاکستان - دبئی ، امارت,True
4,"RT @ZaidZamanHamid: If it is blood clotting, w...",[IN],[],en,0.0,False,2471,لاہور، پاکستان - دبئی ، امارت,True
...,...,...,...,...,...,...,...,...,...
42318,RT @bidemitweets: Dem Ina Ina Ina Ina Ina I kn...,[XX],[],en,0.0,False,661,Abuja,True
42319,RT @HizbAmerica: Help the Muslims of Sri Lanka...,[RU],[],en,0.0,False,23,"Mombasa, Kenya",True
42320,RT @HizbAmerica: Help the Muslims of Sri Lanka...,[RU],[],en,0.0,False,23,"Mombasa, Kenya",True
42321,RT @emrecelikrumi: .@JoeBiden in case you miss...,[TR],"[Erdogan, Israel]",en,0.0,False,18411,Sydney》Brisbane》Melboune》DC》NY,True


In [13]:
df_clean["withheld_anywhere"].describe()

count     42323
unique        2
top        True
freq      41727
Name: withheld_anywhere, dtype: object